In [3]:
# so we can use packages from parent directory
import sys
sys.path.append("..")

In [4]:
# code copied from example experiments.py
import torch
import torch.nn as nn
import numpy as np
from monroe_data import MonroeData, MonroeDataEntry, Color # last two for reading pkl file
import caption_featurizers
from color_featurizers import ColorFeaturizer, color_phi_fourier
from models import LiteralListener, LiteralSpeaker, ImaginativeListener, PragmaticListener, CaptionEncoder, CaptionGenerator, ColorGenerator
from evaluation import score_model, delta_e_dist, Speaker, Score
from experiment import FeatureHandler, evaluate_model

In [5]:
from functools import partial
from scipy import stats

In [4]:
import importlib
import experiment
importlib.reload(experiment)
from experiment import FeatureHandler, evaluate_model

In [6]:
prefix="../"
train_data = MonroeData(prefix + "data/csv/train_corpus_monroe.csv", prefix + "data/entries/train_entries_monroe.pkl")
dev_data_synth  = MonroeData(prefix + "data/csv/dev_corpus_synth_10fold.csv", prefix + "data/entries/dev_corpus_synth_10fold.pkl")

In [7]:
test_data_synth  = MonroeData(prefix + "data/csv/test_corpus_synth_10fold.csv", prefix + "data/entries/test_corpus_synth_10fold.pkl")

In [8]:
def composite_score(eval_df, speaker="gameid"):
    mean_scores = eval_df.groupby(speaker).numOutcome.mean()
    mean_numCleanWords = eval_df.groupby(speaker).numCleanWords.mean()
    mean_clkTime = eval_df.groupby(speaker).clkTime.mean()
    true_scores = mean_scores / mean_clkTime / mean_numCleanWords
    max_score = true_scores.max()
    true_scores /= max_score # normalize the scores
    return true_scores

In [46]:
# 4. Imaginative Listener
def imaginative_listener(model_file="../model/imaginative_listener_with_distractors_linear100hd5epoch_GLOVE_MSE.params"):
    print("Initializing featurizers")
    caption_phi = caption_featurizers.CaptionFeaturizer(tokenizer=caption_featurizers.EndingTokenizer)
    color_phi = ColorFeaturizer(color_phi_fourier, "rgb", normalized=True)

    def target_color_target(data_entry):
        return np.array(data_entry.colors[0].rgb_norm)

    feature_handler = FeatureHandler(train_data, test_data_synth, caption_phi, color_phi, target_fn=target_color_target,
                                randomized_colors=False) #using TEST data now :) 

    print("Obtaining training features") # get features even if you're runnning the pretrained model for example
    #train_features = feature_handler.train_features()
    #train_targets = feature_handler.train_targets()

    imaginative_model = ImaginativeListener(ColorGenerator, criterion=torch.nn.CosineEmbeddingLoss,
                            optimizer=torch.optim.Adam, lr=0.004, num_epochs=5)

    # Creating model
    MSELossSum = lambda: nn.MSELoss(reduction='sum') # sorry for this ugliness..... but this is me passing a parameter to the loss func
    imaginative_model = ImaginativeListener(ColorGenerator, criterion=MSELossSum,
                                optimizer=torch.optim.Adam, lr=0.001, num_epochs=5, use_color=True)
    imaginative_model.init_model(embed_dim=100, hidden_dim=50, vocab_size=feature_handler.caption_featurizer.caption_indexer.size,
                    color_in_dim=54, color_hidden_dim=50, weight_matrix=caption_featurizers.get_pretrained_glove(feature_handler.caption_featurizer.caption_indexer.idx2word.items(), 100, prefix=True))

    imaginative_model.load_model(model_file)
        
    print("Evaluating model")
    output_to_score_de = lambda outputs, targets: np.array([delta_e_dist(outputs[i], targets[i]) for i in range(len(targets))])
    # we want to score based on the model's predictions at the TARGET indices not listener clicked indices,
    # so we change the feature_handler's target function to do that:
    my_score_model = partial(score_model, speaker=Speaker.BY_GAME_ID_COND, return_df=True, score=Score.COMPOSITE)
    result = evaluate_model(test_data_synth, feature_handler, imaginative_model, output_to_score_de, my_score_model, accuracy=False)
    
    return result


In [51]:
def evaluate_imaginative_listener_samples():
    model_directory = "../imaginative_listener_samples"
    num_samples = 10
    aggregate_correlations = []
    close_correlations = []
    split_correlations = []
    far_correlations = []

    aggregate_accuracies = []
    close_accuracies = []
    split_accuracies = []
    far_accuracies = []

    for i in range(num_samples):
        print("Evaluating sample #{}".format(i))
        _, imaginative_listener_eval = imaginative_listener("{}/sample_{}.params".format(model_directory, i))
        il_true_scores = imaginative_listener_eval.groupby('gameid').numOutcome.mean()
        il_model_scores = imaginative_listener_eval.groupby('gameid').model_scores.mean()
        il_true_scores_composite = composite_score(imaginative_listener_eval)

        aggregate_correlations.append(stats.pearsonr(il_model_scores, il_true_scores_composite))
        # arbitrarily say we get it right if we assign a majority of the probability mass to it
        #aggregate_accuracies.append(sum(imaginative_listener_eval.model_scores > 0.5)/len(imaginative_listener_eval.model_scores))
        aggregate_accuracies.append(np.mean(imaginative_listener_eval.model_scores))

        # separate out conditions
        imaginative_listener_close = imaginative_listener_eval[imaginative_listener_eval.condition == "close"]
        imaginative_listener_split = imaginative_listener_eval[imaginative_listener_eval.condition == "split"]
        imaginative_listener_far = imaginative_listener_eval[imaginative_listener_eval.condition == "far"]

        imaginative_listener_close_true_scores = imaginative_listener_close.groupby('gameid').numOutcome.mean()
        imaginative_listener_close_model_scores = imaginative_listener_close.groupby('gameid').model_scores.mean()

        imaginative_listener_split_true_scores =  imaginative_listener_split.groupby('gameid').numOutcome.mean()
        imaginative_listener_split_model_scores = imaginative_listener_split.groupby('gameid').model_scores.mean()

        imaginative_listener_far_true_scores =  imaginative_listener_far.groupby('gameid').numOutcome.mean()
        imaginative_listener_far_model_scores = imaginative_listener_far.groupby('gameid').model_scores.mean()

        # turn true scores to composite, gricean scores
        imaginative_listener_close_composite = composite_score(imaginative_listener_close)
        imaginative_listener_split_composite = composite_score(imaginative_listener_split)
        imaginative_listener_far_composite   = composite_score(imaginative_listener_far)

        close_correlations.append(stats.pearsonr(imaginative_listener_close_composite, imaginative_listener_close_model_scores))
        split_correlations.append(stats.pearsonr(imaginative_listener_split_composite, imaginative_listener_split_model_scores))
        far_correlations.append(stats.pearsonr(imaginative_listener_far_composite, imaginative_listener_far_model_scores))

    #     close_accuracies.append(sum(imaginative_listener_close_model_scores > 0.5)/len(imaginative_listener_close_model_scores))
    #     split_accuracies.append(sum(imaginative_listener_split_model_scores > 0.5)/len(imaginative_listener_split_model_scores))
    #     far_accuracies.append(sum(imaginative_listener_far_model_scores > 0.5)/len(imaginative_listener_far_model_scores))
        close_accuracies.append(np.mean(imaginative_listener_close_model_scores))
        split_accuracies.append(np.mean(imaginative_listener_split_model_scores))
        far_accuracies.append(np.mean(imaginative_listener_far_model_scores))
        
        print("Most recent stats:")
        print("agg acc:", aggregate_accuracies[-1])
        print("clo acc:", close_accuracies[-1])
        print("spl acc:", split_accuracies[-1])
        print("far acc:", far_accuracies[-1])
        print("agg cor:", aggregate_correlations[-1])
        print("clo cor:", close_correlations[-1])
        print("spl cor:", split_correlations[-1])
        print("far cor:", far_correlations[-1])
        
    return {"aggregate_accuracies": aggregate_accuracies,
            "close_accuracies": close_accuracies,
            "split_accuracies": split_accuracies,
            "far_accuracies": far_accuracies,
            "aggregate_correlations": aggregate_correlations,
            "close_correlations":close_correlations,
            "far_correlations":far_correlations,
            "split_correlations": split_correlations}


In [52]:
results = evaluate_imaginative_listener_samples()

Evaluating sample #0
Initializing featurizers
Obtaining training features
Evaluating model
Got here to composite score
Most recent stats:
agg acc: 23.98516954890258
clo acc: 18.27353323876809
spl acc: 19.544536012224782
far acc: 34.14159079769889
agg cor: (-0.886422302978764, 4.250960310734327e-178)
clo cor: (-0.455535609378521, 2.0635307187084423e-28)
spl cor: (-0.5473674197436782, 1.3343968892457512e-42)
far cor: (-0.8657290229509013, 3.1337385241772896e-160)
Evaluating sample #1
Initializing featurizers
Obtaining training features
Evaluating model
Got here to composite score
Most recent stats:
agg acc: 24.211091965944536
clo acc: 18.65899781108387
spl acc: 19.767035392086786
far acc: 34.21259842311515
agg cor: (-0.8860109120142069, 1.0393073393501362e-177)
clo cor: (-0.44432579193215566, 5.887339051429764e-27)
spl cor: (-0.5267565591108372, 4.9684543265154734e-39)
far cor: (-0.8675670970509768, 1.079527020742103e-161)
Evaluating sample #2
Initializing featurizers
Obtaining training 

In [53]:
import pickle
with open("../results/imaginative_listener_assessment.pkl", "wb") as file:
    pickle.dump(results, file)

In [57]:
np.mean([cor[0] for cor in results['aggregate_correlations']])

-0.8857654592635387

In [33]:
aggregate_accuracies

[24.225927799225445]

In [34]:
close_accuracies

[18.50006086024096]

In [35]:
far_accuracies

[34.470457996575014]

In [36]:
split_accuracies

[19.710338275301137]

In [37]:
aggregate_correlations

[(-0.8799177945947197, 1.4376357425904765e-168)]

In [38]:
close_correlations

[(-0.4409533567945598, 5.226923223263904e-26)]

In [39]:
far_correlations

[(-0.8577761621065179, 5.865956224847408e-151)]

In [40]:
split_correlations

[(-0.5079681530415953, 2.896292325793216e-35)]

In [15]:
# 1. Literal Listener
# -----------------------------------------
# TODO: FILL IN PARAMETERS 
def literal_listener_experiment(train=False, evaluate=True, epochs=5, embed_dim = 100, hidden_dim = 100, color_dim= 54, model_file="../model/literal_listener_5epoch-2.params"):

    # Initializing featurizers
    print("Initializing featurizers")
    caption_phi = caption_featurizers.CaptionFeaturizer(tokenizer=caption_featurizers.EndingTokenizer) # Use with parameter files that end in `endings_tkn`
    # caption_phi = caption_featurizers.CaptionFeaturizer(tokenizer=caption_featurizers.WhitespaceTokenizer) # Use with parameter files don't
    color_phi = ColorFeaturizer(color_phi_fourier, "rgb", normalized=True)
    feature_handler = FeatureHandler(train_data, test_data_synth, caption_phi, color_phi) # target function is initialized by default

    print("Initializing model")
    model = LiteralListener(CaptionEncoder, num_epochs = epochs)
    model.init_model(embed_dim = embed_dim, hidden_dim = hidden_dim, vocab_size = feature_handler.caption_featurizer.caption_indexer.size,
                 color_dim = color_dim)

    model.load_model(model_file)

    if not evaluate:
        return model
    else:
        # convert the model output to a score for that particular round
        print("Evaluating model")
        output_to_score = lambda model_outputs, targets: np.exp(model_outputs[np.arange(len(model_outputs)), targets]) # get the model's predicted probablity at each target index and use that as the score
        my_score_model = partial(score_model, speaker=Speaker.BY_GAME_ID_COND, return_df=True, score=Score.COMPOSITE)
        return evaluate_model(test_data_synth, feature_handler, model, output_to_score, my_score_model, accuracy=False)

In [46]:
# I assume pragmatic will either be similar or need its own type of thing
def evaluate_literal_listener_samples():
    model_directory # FILL THIS IN = "../literal_listener_samples"
    num_samples = 10
    aggregate_correlations = []
    close_correlations = []
    split_correlations = []
    far_correlations = []

    aggregate_accuracies = []
    close_accuracies = []
    split_accuracies = []
    far_accuracies = []


    for i in range(num_samples):
        _, listener_eval = literal_listener("{}/sample_{}.params".format(model_directory, i))
        true_scores = listener_eval.groupby('gameid').numOutcome.mean()
        model_scores = listener_eval.groupby('gameid').model_scores.mean()
        true_scores_composite = composite_score(listener_eval)

        aggregate_correlations.append(stats.pearsonr(model_scores, true_scores_composite))
        # arbitrarily say we get it right if we assign a majority of the probability mass to it
        aggregate_accuracies.append(sum(listener_eval.model_scores > 0.5)/len(listener_eval.model_scores))

        # separate out conditions
        listener_close = listener_eval[listener_eval.condition == "close"]
        listener_split = listener_eval[listener_eval.condition == "split"]
        listener_far =   listener_eval[listener_eval.condition == "far"]

        listener_close_true_scores =  listener_close.groupby('gameid').numOutcome.mean()
        listener_close_model_scores = listener_close.groupby('gameid').model_scores.mean()

        listener_split_true_scores =  listener_split.groupby('gameid').numOutcome.mean()
        listener_split_model_scores = listener_split.groupby('gameid').model_scores.mean()

        listener_far_true_scores =  listener_far.groupby('gameid').numOutcome.mean()
        listener_far_model_scores = listener_far.groupby('gameid').model_scores.mean()

        # turn true scores to composite, gricean scores
        listener_close_composite = composite_score(listener_close)
        listener_split_composite = composite_score(listener_split)
        listener_far_composite   = composite_score(listener_far)

        close_correlations.append(stats.pearsonr(listener_close_composite, listener_close_model_scores))
        split_correlations.append(stats.pearsonr(listener_split_composite, listener_split_model_scores))
        far_correlations.append(stats.pearsonr(listener_far_composite,     listener_far_model_scores))

        close_accuracies.append(sum(listener_close_model_scores > 0.5)/len(listener_close_model_scores))
        split_accuracies.append(sum(listener_split_model_scores > 0.5)/len(listener_split_model_scores))
        far_accuracies.append(sum(  listener_far_model_scores > 0.5)/len(listener_far_model_scores))

    return {"aggregate_accuracies": aggregate_accuracies,
            "close_accuracies": close_accuracies,
            "split_accuracies": split_accuracies,
            "far_accuracies": far_accuracies,
            "aggregate_correlations": aggregate_correlations,
            "close_correlations":close_correlations,
            "far_correlations":far_correlations,
            "split_correlations": split_correlations}


In [14]:
def literal_speaker_experiment(train=False, epochs=5, color_in_dim = 54, color_dim = 100, embed_dim = 100, hidden_dim = 100, lr = 0.004, model_file="model/literal_speaker_5epoch.params"):
    # Initializing featurizers
    print("Initializing featurizers")
    caption_phi = caption_featurizers.CaptionFeaturizer(tokenizer=caption_featurizers.EndingTokenizer)  # we'll use the EndingTokenizer in order to have common training data, but should technically be WhitespaceTokenizer

    color_phi = ColorFeaturizer(color_phi_fourier, "hsv", normalized=True) # speaker uses hsv 
    
    # speaker's target is to predict tokens following the SOS token
    def speaker_target(data_entry):
        _, caption_ids = caption_phi.to_string_features(data_entry.caption) # this probably works...
        target = caption_ids[1:]
        return target

    feature_handler = FeatureHandler(train_data, test_data_synth, caption_phi, color_phi, target_fn=speaker_target, randomized_colors=False)

    print("Obtaining training features")
    train_features = feature_handler.train_features()
    train_targets = feature_handler.train_targets()

    print("Initializing model")
    model = LiteralSpeaker(CaptionGenerator, optimizer=torch.optim.Adam, lr=lr, num_epochs=epochs)
    #lit_speaker = Speaker(color_embed_dim, caption_phi.caption_indexer.size, embed_dim, hidden_dim)
    model.init_model(color_in_dim=color_in_dim, color_dim=color_dim,
                                  vocab_size=caption_phi.caption_indexer.size, embed_dim=embed_dim,
                                 speaker_hidden_dim=hidden_dim)
    if train:
        print("Training model and saving to {}:".format(model_file))
        model.fit(train_features, train_targets)
        model.save_model(model_file)
    else:
        print("Loading pretrained model")
        model.load_model(model_file)

    return model

In [11]:
def pragmatic_listener(evaluate=True,
        literal_model_file="../model/best_literal_listener.params",
        speaker_model_file="../model/literal_speaker_30epochGLOVE.params",
        alpha=0.5, # best
        composite=True):
    color_phi_listener = ColorFeaturizer(color_phi_fourier, "rgb", normalized=True)
    color_phi_speaker = ColorFeaturizer(color_phi_fourier, "hsv", normalized=True)
    caption_phi = caption_featurizers.CaptionFeaturizer(tokenizer=caption_featurizers.EndingTokenizer)
    feature_handler_listener = FeatureHandler(train_data, test_data_synth, caption_phi, color_phi_listener)
    feature_handler_speaker = FeatureHandler(train_data, test_data_synth, caption_phi, color_phi_speaker)
    print("Loading literal listener")
    listener_model = literal_listener_experiment(train=False, evaluate=False, model_file=literal_model_file)
    print("Loading literal speaker")
    speaker_model = literal_speaker_experiment(train=False, model_file=speaker_model_file)
    pragmatic_model = PragmaticListener(listener_model, speaker_model,alpha=alpha)
    if not evaluate:
        return pragmatic_model
    else:
        output_to_score = lambda model_outputs, targets: np.exp(model_outputs[np.arange(len(model_outputs)), targets]) # get the model's predicted probablity at each target index and use that as the score
        output_to_score_acc = lambda model_outputs, targets: np.argmax(model_outputs, axis=1) == targets 
        if composite:
            my_score_model = partial(score_model, speaker=Speaker.BY_GAME_ID_COND, return_df=True, score=Score.COMPOSITE)
        else:
            my_score_model = partial(score_model, speaker=Speaker.BY_GAME_ID_COND, return_df=True)
        eval_p = evaluate_model(test_data_synth, feature_handler_listener, pragmatic_model, output_to_score,
                              my_score_model, feature_handler_2=feature_handler_speaker) 
        eval_acc = evaluate_model(test_data_synth, feature_handler_listener, pragmatic_model, output_to_score_acc,
                              my_score_model, feature_handler_2=feature_handler_speaker)
        return (eval_p, eval_acc)

In [21]:
def evaluate_pragmatic_listener_samples():
    num_samples = 10
    aggregate_correlations = []
    close_correlations = []
    split_correlations = []
    far_correlations = []

    aggregate_accuracies = []
    close_accuracies = []
    split_accuracies = []
    far_accuracies = []
    
    for i in range(num_samples):
        listener_eval, listener_eval_acc = pragmatic_listener()
        _, listener_eval = listener_eval
        listener_eval = listener_eval[0] # these are tuples for some reason...
        _, listener_eval_acc = listener_eval_acc
        listener_eval_acc = listener_eval_acc[0]
        true_scores = listener_eval.groupby('gameid').numOutcome.mean()
        model_scores = listener_eval.groupby('gameid').model_scores.mean()
        true_scores_composite = composite_score(listener_eval)

        aggregate_correlations.append(stats.pearsonr(model_scores, true_scores_composite))
        # arbitrarily say we get it right if we assign a majority of the probability mass to it
        aggregate_accuracies.append(sum(listener_eval_acc.model_scores)/len(listener_eval_acc.model_scores))

        # separate out conditions
        listener_close = listener_eval[listener_eval.condition == "close"]
        listener_split = listener_eval[listener_eval.condition == "split"]
        listener_far =   listener_eval[listener_eval.condition == "far"]
        
        listener_close_acc = listener_eval_acc[listener_eval_acc.condition == "close"]
        listener_split_acc = listener_eval_acc[listener_eval_acc.condition == "split"]
        listener_far_acc =   listener_eval_acc[listener_eval_acc.condition == "far"]

        listener_close_true_scores =  listener_close.groupby('gameid').numOutcome.mean()
        listener_close_model_scores = listener_close.groupby('gameid').model_scores.mean()

        listener_split_true_scores =  listener_split.groupby('gameid').numOutcome.mean()
        listener_split_model_scores = listener_split.groupby('gameid').model_scores.mean()

        listener_far_true_scores =  listener_far.groupby('gameid').numOutcome.mean()
        listener_far_model_scores = listener_far.groupby('gameid').model_scores.mean()

        # turn true scores to composite, gricean scores
        listener_close_composite = composite_score(listener_close)
        listener_split_composite = composite_score(listener_split)
        listener_far_composite   = composite_score(listener_far)

        close_correlations.append(stats.pearsonr(listener_close_composite, listener_close_model_scores))
        split_correlations.append(stats.pearsonr(listener_split_composite, listener_split_model_scores))
        far_correlations.append(stats.pearsonr(listener_far_composite,     listener_far_model_scores))

        close_accuracies.append(sum(listener_close_acc.model_scores)/len(listener_close_acc.model_scores))
        split_accuracies.append(sum(listener_split_acc.model_scores)/len(listener_split_acc.model_scores))
        far_accuracies.append(sum(  listener_far_acc.model_scores)/len(listener_far_acc.model_scores))
        
        print({"aggregate_accuracies": aggregate_accuracies,
            "close_accuracies": close_accuracies,
            "split_accuracies": split_accuracies,
            "far_accuracies": far_accuracies,
            "aggregate_correlations": aggregate_correlations,
            "close_correlations":close_correlations,
            "far_correlations":far_correlations,
            "split_correlations": split_correlations})

    return {"aggregate_accuracies": aggregate_accuracies,
            "close_accuracies": close_accuracies,
            "split_accuracies": split_accuracies,
            "far_accuracies": far_accuracies,
            "aggregate_correlations": aggregate_correlations,
            "close_correlations":close_correlations,
            "far_correlations":far_correlations,
            "split_correlations": split_correlations}


In [22]:
%%time
results = evaluate_pragmatic_listener_samples()
results

Loading literal listener
Initializing featurizers
Initializing model
Loading literal speaker
Initializing featurizers
Obtaining training features
Initializing model
Loading pretrained model
Got here to composite score
Accuracy: 0.4700757575757576
Got here to composite score
Accuracy: 0.46825757575757576
{'aggregate_accuracies': [0.46825757575757576], 'close_accuracies': [0.4481818181818182], 'split_accuracies': [0.47306818181818183], 'far_accuracies': [0.48352272727272727], 'aggregate_correlations': [(0.9617727835682962, 4.883427095399908e-298)], 'close_correlations': [(0.8847172825736496, 1.690239250200163e-176)], 'far_correlations': [(0.9484928640127243, 9.463052936984235e-265)], 'split_correlations': [(0.9041780328750628, 1.8758836709584286e-196)]}
Loading literal listener
Initializing featurizers
Initializing model
Loading literal speaker
Initializing featurizers
Obtaining training features
Initializing model
Loading pretrained model
Got here to composite score
Accuracy: 0.46939393

Loading literal listener
Initializing featurizers
Initializing model
Loading literal speaker
Initializing featurizers
Obtaining training features
Initializing model
Loading pretrained model
Got here to composite score
Accuracy: 0.4693181818181818
Got here to composite score
Accuracy: 0.46950757575757573
{'aggregate_accuracies': [0.46825757575757576, 0.4700378787878788, 0.46954545454545454, 0.46893939393939393, 0.46856060606060607, 0.4693181818181818, 0.46950757575757573], 'close_accuracies': [0.4481818181818182, 0.4526136363636364, 0.45125, 0.44977272727272727, 0.4501136363636364, 0.4510227272727273, 0.45125], 'split_accuracies': [0.47306818181818183, 0.47306818181818183, 0.47352272727272726, 0.4727272727272727, 0.4715909090909091, 0.4732954545454545, 0.4727272727272727], 'far_accuracies': [0.48352272727272727, 0.4844318181818182, 0.4838636363636364, 0.4843181818181818, 0.4839772727272727, 0.48363636363636364, 0.48454545454545456], 'aggregate_correlations': [(0.9617727835682962, 4.8834

In [23]:
results

{'aggregate_accuracies': [0.46825757575757576,
  0.4700378787878788,
  0.46954545454545454,
  0.46893939393939393,
  0.46856060606060607,
  0.4693181818181818,
  0.46950757575757573,
  0.4694318181818182,
  0.46912878787878787,
  0.47015151515151515],
 'aggregate_correlations': [(0.9617727835682962, 4.883427095399908e-298),
  (0.9619061957556335, 1.9820351122321373e-298),
  (0.9620535235642376, 7.294941110426085e-299),
  (0.9618692907112215, 2.5443736796406772e-298),
  (0.9616145656342249, 1.4169314207903172e-297),
  (0.9617795779256101, 4.664601104499242e-298),
  (0.9617489822752973, 5.733814014436458e-298),
  (0.9618894272250932, 2.2202909668708005e-298),
  (0.9618884771641455, 2.2346131807101505e-298),
  (0.9619051200765443, 1.9965235853990405e-298)],
 'close_accuracies': [0.4481818181818182,
  0.4526136363636364,
  0.45125,
  0.44977272727272727,
  0.4501136363636364,
  0.4510227272727273,
  0.45125,
  0.4505681818181818,
  0.45227272727272727,
  0.45227272727272727],
 'close_corre

In [24]:
import pickle
with open("../results/pragmatic_listener_assessment_accuracy.pkl", "wb") as file:
    pickle.dump(results, file)

In [62]:
np.mean([cor[0] for cor in results['aggregate_correlations']])

0.961719739609744

In [25]:
# second time
np.mean([cor[0] for cor in results['aggregate_correlations']])

0.9618427943900304

In [63]:
np.mean([cor[0] for cor in results['close_correlations']])

0.8845222428859618

In [64]:
np.mean([cor[0] for cor in results['far_correlations']])

0.9485555018949304

In [65]:
np.mean([cor[0] for cor in results['split_correlations']])

0.9046507905374798

In [68]:
np.mean([cor for cor in results['aggregate_accuracies']])

0.45305303030303035

In [26]:
# second time
np.mean([cor for cor in results['aggregate_accuracies']])

0.46928787878787875

In [20]:
_, b = (1, )

ValueError: not enough values to unpack (expected 2, got 1)